In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'co2': 0.0008}
band = [11]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [3, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.75), (1.75, 1.6, 1.85)]
  wgt = [(0.7, 0.8, 0.7), (0.8, 0.7, 0.8)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.006631                    1  0.194542
0.000750        2 -0.204027                    2 -0.275384
0.001052        3 -0.143888                    3 -0.199153
0.001476        4 -0.114543                    4 -0.171510
0.002070        5 -0.090078                    5 -0.158889
0.002904        6 -0.066743                    6 -0.143023
0.004074        7 -0.044360                    7 -0.120871
0.005714        8 -0.023498                    8 -0.084983
0.008015        9 -0.004512                    9 -0.042470
0.011243       10  0.012791                   10  0.005658
0.015771       11  0.029365                   11  0.057694
0.022122       12  0.047370                   12  0.111805
0.031031       13  0.074847                   13  0.169174
0.043528       14  0.095722                   14  0.203640
0.061057       15  0.093199                   15  0.191496
0.085645       16  0.088375                   16  0.155082
0.120136       17  0.091416                   17  0.118943
0.168516       18  0.098312                   18  0.093539
0.236378       19  0.109414                   19  0.084399
0.331549       20  0.129083                   20  0.096210
0.465100       21  0.177535                   21  0.151607
0.652400       22  0.161452                   22  0.148569
0.915100       23  0.084205                   23  0.092849
1.283650       24  0.041385                   24  0.062791
1.800600       25  0.022596                   25  0.042853
2.525700       26  0.011854                   26  0.023434
3.542800       27  0.004743                   27  0.006308
4.969550       28  0.001593                   28 -0.002160
6.970850       29  0.001294                   29 -0.000201
9.778100       30  0.001061                   30  0.002717
13.715850      31  0.000412                   31  0.002447
19.239350      32 -0.000358                   32  0.000749
26.987250      33 -0.000415                   33 -0.000266
37.855300      34 -0.000014                   34 -0.000145
53.100050      35  0.000083                   35 -0.000070
73.887500      36  0.000100                   36  0.000015
97.662500      37  0.000109                   37  0.000123
121.437500     38 -0.000067                   38  0.000012
145.212500     39 -0.000293                   39 -0.000180
168.987500     40 -0.000426                   40 -0.000293
192.762500     41 -0.000547                   41 -0.000428
216.537500     42 -0.000671                   42 -0.000595
240.312500     43 -0.000811                   43 -0.000798
264.087500     44 -0.001024                   44 -0.001088
287.862500     45 -0.001502                   45 -0.001669
311.637500     46 -0.001829                   46 -0.002063
335.412500     47 -0.001622                   47 -0.001893
359.187500     48 -0.001350                   48 -0.001616
382.962500     49 -0.001232                   49 -0.001475
406.737500     50 -0.001127                   50 -0.001317
430.512500     51 -0.001008                   51 -0.001123
454.287500     52 -0.000866                   52 -0.000949
478.062500     53 -0.000699                   53 -0.000681
501.837500     54 -0.000503                   54 -0.000427
525.612500     55 -0.000277                   55 -0.000140
549.387500     56 -0.000024                   56  0.000170
573.162500     57  0.000250                   57  0.000496
596.937500     58  0.000536                   58  0.000837
620.712500     59  0.000821                   59  0.001085
644.487500     60  0.001094                   60  0.001421
668.262500     61  0.001345                   61  0.001630
692.037500     62  0.001574                   62  0.001812
715.812500     63  0.001782                   63  0.001972
739.587500     64  0.001984                   64  0.002139
763.362500     65  0.002195                   65  0.002346
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -1.239651  0.000000 -1.239651 -1.205042  9.129459e-09   
0.000624    2     -1.239672  0.000021 -1.239651 -1.205070  4.219497e-05   
0.000876    3     -1.239682  0.000025 -1.239657 -1.205085  4.919738e-05   
0.001229    4     -1.239697  0.000034 -1.239663 -1.205105  6.087970e-05   
0.001723    5     -1.239716  0.000046 -1.239670 -1.205131  7.711472e-05   
0.002417    6     -1.239741  0.000064 -1.239677 -1.205166  9.832966e-05   
0.003391    7     -1.239774  0.000090 -1.239685 -1.205210  1.261722e-04   
0.004757    8     -1.239818  0.000126 -1.239692 -1.205267  1.641476e-04   
0.006672    9     -1.239875  0.000178 -1.239697 -1.205341  2.186815e-04   
0.009359    10    -1.239949  0.000250 -1.239699 -1.205435  2.989602e-04   
0.013128    11    -1.240042  0.000350 -1.239693 -1.205553  4.191629e-04   
0.018415    12    -1.240161  0.000486 -1.239675 -1.205698  6.001949e-04   
0.025830    13    -1.240306  0.000673 -1.239633 -1.205870  8.711448e-04   
0.036232    14    -1.240467  0.000927 -1.239541 -1.206059  1.268255e-03   
0.050823    15    -1.240631  0.001256 -1.239375 -1.206249  1.809871e-03   
0.071291    16    -1.240804  0.001655 -1.239149 -1.206446  2.471933e-03   
0.100000    17    -1.240998  0.002149 -1.238849 -1.206661  3.213856e-03   
0.140271    18    -1.241203  0.002790 -1.238413 -1.206880  4.000943e-03   
0.196760    19    -1.241394  0.003639 -1.237755 -1.207079  4.825865e-03   
0.275997    20    -1.241509  0.004781 -1.236728 -1.207213  5.752216e-03   
0.387100    21    -1.241366  0.006337 -1.235029 -1.207134  6.939628e-03   
0.543100    22    -1.240174  0.008426 -1.231748 -1.205970  8.577273e-03   
0.761700    23    -1.237749  0.010182 -1.227567 -1.203549  1.000363e-02   
1.068500    24    -1.235198  0.010690 -1.224507 -1.200990  1.081963e-02   
1.498800    25    -1.232929  0.010532 -1.222398 -1.198643  1.167323e-02   
2.102400    26    -1.230921  0.010139 -1.220782 -1.196453  1.254715e-02   
2.949000    27    -1.229179  0.009586 -1.219593 -1.194440  1.288522e-02   
4.136600    28    -1.227821  0.008895 -1.218926 -1.192750  1.208281e-02   
5.802500    29    -1.226861  0.008249 -1.218612 -1.191478  1.038378e-02   
8.139200    30    -1.226097  0.007843 -1.218254 -1.190402  9.252932e-03   
11.417000   31    -1.225433  0.007591 -1.217842 -1.189417  9.322965e-03   
16.014700   32    -1.224967  0.007350 -1.217618 -1.188669  9.907396e-03   
22.464000   33    -1.225061  0.007170 -1.217891 -1.188603  1.041388e-02   
31.510500   34    -1.225865  0.007529 -1.218336 -1.189405  1.093114e-02   
44.200100   35    -1.227110  0.008754 -1.218357 -1.190687  1.199552e-02   
62.000000   36    -1.228776  0.010595 -1.218181 -1.192328  1.348945e-02   
85.775000   37    -1.230765  0.012866 -1.217899 -1.194181  1.538380e-02   
109.550000  38    -1.232486  0.014893 -1.217593 -1.195745  1.729461e-02   
133.325000  39    -1.234138  0.016356 -1.217782 -1.197253  1.883699e-02   
157.100000  40    -1.235995  0.017390 -1.218606 -1.199059  2.013563e-02   
180.875000  41    -1.238082  0.018277 -1.219805 -1.201207  2.145816e-02   
204.650000  42    -1.240419  0.019073 -1.221347 -1.203740  2.278636e-02   
228.425000  43    -1.243038  0.019801 -1.223237 -1.206731  2.410130e-02   
252.200000  44    -1.245999  0.020479 -1.225521 -1.210256  2.537869e-02   
275.975000  45    -1.249528  0.021122 -1.228406 -1.214551  2.660925e-02   
299.750000  46    -1.254522  0.021885 -1.232636 -1.220583  2.794078e-02   
323.525000  47    -1.261525  0.023737 -1.237787 -1.228874  3.042073e-02   
347.300000  48    -1.269886  0.027529 -1.242356 -1.238726  3.494065e-02   
371.075000  49    -1.279027  0.032870 -1.246157 -1.249480  4.114255e-02   
394.850000  50    -1.288871  0.039244 -1.249628 -1.261040  4.854834e-02   
418.625000  51    -1.299386  0.046583 -1.252803 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')